In [2]:
import pandas as pd
import datetime
import numpy as np
import altair as alt

データ量が多くなると動作が重くなるため、整形済みデータをアップして負荷を減らす  
→ dfをAddDis調製データ.xlsxとしてアップ

In [6]:
# マスタ修正の必要性を確認
rawData = 'AddDis調製rawデータ.xlsx'
mst = 'C:\\Users\\hmill\\OneDrive\\デスクトップ\\sample_data\\データの可視化\\化療調製_AddDisデータ分析\\mst_data.xlsx'
df = pd.read_excel(rawData)
krebs = pd.read_excel(mst, sheet_name='krebs')
krebs_cds = krebs['薬品コード'].to_list()
ex = pd.read_excel(mst, sheet_name='exclusion')
exclusion = ex['薬品コード'].to_list()
ph = pd.read_excel(mst, sheet_name='ph')
ph_dict = {}
for i in range(len(ph)):
    key = ph.iloc[i, 0]
    value = ph.iloc[i, 1]
    ph_dict[f'{key}'] = value
ph_list = ph['調製者'].to_list()

df_check = df[['薬品コード', '薬品名', '調製者']]
df_drugs = df_check.query('薬品コード not in @krebs_cds')
df_drugs = df_drugs.query('薬品コード not in @exclusion')

if len(df_drugs) == 0:
  print('薬品マスタ（krebs・除外）の修正は不要です')
else:
  df_drugs.to_excel('mstメンテナンス\\薬品マスタ追加リスト.xlsx', index=False)
  print('薬品マスタ（krebs・除外）への追加リストを出力しました')

df_ph = df_check.query('調製者 not in @ph_list')
if len(df_ph) == 0:
  print('薬剤師マスタの修正は不要です')
else:
  df_ph.to_excel(f'mstメンテナンス\\薬剤師マスタ追加リスト.xlsx', index=False)
  print('薬剤師マスタへの追加リストを出力しました')

薬品マスタ（krebs・除外）の修正は不要です
薬剤師マスタの修正は不要です


In [7]:
# str日付をdatetimeに変換
df['実施日'] = pd.to_datetime(df['実施日'])
for i in range(len(df)):
    dt = df.loc[i, '実施日']
    h1, m1, s1 = df.loc[i, '調製開始'].split(':')
    df.loc[i, '調製開始'] = dt + datetime.timedelta(hours=int(h1), minutes=int(m1), seconds=int(s1))
    h2, m2, s2 = df.loc[i, '調製時間'].split(':')
    df.loc[i, '調製時間'] = dt + datetime.timedelta(minutes=int(m2), seconds=int(s2))

# 不要な行・列を削除  
df1 = df.query('薬品コード != @exclusion')
df1 = df1[['実施日', '入外', '科名', '病棟', 'オーダー番号', '調製者', '薬品本数', '薬品コード', '薬品名', '調製開始', '調製時間', '保留時間',]]
df1.sort_values('オーダー番号', ascending=True, inplace=True)

#オーダ番号がないレコードは削除
df1.dropna(subset='オーダー番号', inplace=True)
df1 = pd.merge(df1, krebs, how='inner')

# オーダー番号と用量（合計）の紐づけ
df1['mg'] = df1['薬品本数'] * df1['contain']

dose = df1.pivot_table(index='オーダー番号', values='mg', aggfunc='sum')
dose.reset_index(inplace=True)

# 調製時間の算出
prep_times = []
for i in range(len(df1)):
    tdelta = df1.loc[i, '調製時間'] - df1.loc[i, '実施日']
    sec = tdelta.total_seconds()  #秒に換算
    prep_times.append(sec/60)
df1['prep_time'] = prep_times
df = df1[['実施日', '入外', '科名', '病棟', 'オーダー番号', '薬品本数', '薬品コード', '薬品名', '調製開始', 'stem', 'contain', 'prep_time', 
          '調製者'
          ]]
df.sort_values('オーダー番号', ascending=False, inplace=True)  #同オーダー番号の最大値を採用
df.drop_duplicates(subset='オーダー番号', keep='first', inplace=True)
df = pd.merge(df, dose, how='outer')

hours = []
for i in range(len(df)):
  hour = df.loc[i, '調製開始'].strftime('%H時')
  hours.append(hour)
df['hour'] = hours

wd = []
for i in range(len(df)):
  weekday = df.loc[i, '実施日'].strftime('%a')
  wd.append(weekday)
df['wd'] = wd

for i in range(len(df)):
  key = df.loc[i, '調製者']
  df.loc[i, '調製者'] = ph_dict[key]

C:\Users\hmill\AppData\Local\Temp\ipykernel_11384\3298755662.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values('オーダー番号', ascending=False, inplace=True)  #同オーダー番号の最大値を採用
C:\Users\hmill\AppData\Local\Temp\ipykernel_11384\3298755662.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset='オーダー番号', keep='first', inplace=True)


In [9]:
df.to_excel('AddDis調製データ.xlsx', index=False)